In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import recall_score,precision_score,f1_score               

import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
import scipy
from keras import backend as K
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Dropout,Input,Embedding,RepeatVector,Permute,merge,Lambda
from keras.layers import multiply ,concatenate ,subtract,add,Add,Reshape                   
from keras.optimizers import RMSprop ,Adam
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Hike/train/user_features.csv')

In [3]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Hike/test.csv')

Checking if all ids in test are present in train_feat

In [5]:
train_data_new = train_data

In [6]:
train_data1 = train_data_new[['node1_id']]
train_data2 = train_data_new[['node2_id']]

y_train = train_data_new['is_chat']

In [7]:
test_data1 = test_data[['node1_id']]
test_data2 = test_data[['node2_id']]


In [8]:
train_data.shape

(70661802, 3)

In [9]:
train_data_new =train_data[0:7066180]

Bringing the Train Features

In [48]:
train_feat1 =  pd.merge(left = train_data_new, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [49]:
train_feat2 = pd.merge(left = train_data_new, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Bringing Test Features

In [56]:
test_feat1 = pd.merge(left = test_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [57]:
test_feat2 = pd.merge(left = test_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Standardizing Columns

In [22]:
cols_std = ['f1','f2','f3','f4','f5', 'f6','f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']

In [50]:
train_feat1 = train_feat1[cols_std]
train_feat2 = train_feat2[cols_std]


In [51]:
train_feat1 = pd.get_dummies(data= train_feat1, columns= cols_std)

In [54]:
train_feat2 = pd.get_dummies(data= train_feat2, columns= cols_std)

In [58]:
test_feat1 = test_feat1[cols_std]
test_feat2 = test_feat2[cols_std]

In [59]:
test_feat1 = pd.get_dummies(data= test_feat1, columns= cols_std)

In [60]:
test_feat2 = pd.get_dummies(data= test_feat2, columns= cols_std)

In [61]:
y_train = train_data_new['is_chat']

Baseline Random Forest


In [62]:
from sklearn.ensemble import RandomForestClassifier 

In [63]:
x_train = pd.concat([train_feat1, train_feat2], axis=1)

In [65]:
x_test = pd.concat([test_feat1, test_feat2], axis=1)

In [66]:
from sklearn.model_selection import train_test_split
x1_train, x1_val, y1_train, y1_val = train_test_split(x_train, y_train,test_size=0.2,random_state=0)

In [67]:
rf =  RandomForestClassifier(bootstrap=True, criterion='gini',
                max_depth=20, max_features='auto', max_leaf_nodes=None, 
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=6,
                oob_score=False, random_state=12, warm_start=False,
                class_weight={0:5,1:80}, verbose=1)

In [ ]:
rf.fit(x1_train, y1_train)

In [ ]:
print (rf.score(x1_val, y1_val))

In [ ]:
print (recall_score(y1_val, rf.predict(x1_val)))

In [ ]:
print (precision_score(y1_val, rf.predict(x1_val)))

In [ ]:
#y1_train[y1_train==0].shape

In [ ]:
rf_pred = rf.predict_proba(x_test)

In [ ]:
#rf_pred[0:100]

In [ ]:
y_ones = rf_pred[:,1]

In [ ]:
sub_df = test_data.copy()

In [ ]:
sub_df['is_chat'] = y_ones

In [ ]:
sub_df = sub_df[['id','is_chat']]

In [ ]:
z = np.load('/Users/s0c02nj/Desktop/Hike/same_node_ids.npy')

In [ ]:
#sub_df

In [ ]:
z_set = set(z)

In [ ]:
out = []
for i in tqdm(range(0,len(sub_df))):
    
    i_id = sub_df['id'].iloc[i]
    
    if i_id in z_set:
        k=0
    else:
        k = sub_df['is_chat'].iloc[i]
        
    out.append(k)

In [ ]:
sub_df['is_chat'] = out

In [ ]:
sub_df.to_csv('/Users/s0c02nj/Desktop/Hike/Submission.csv',index=False)

In [ ]:
#train_feat1.shape
#y_cat.shape